In [ ]:
'''
Return variables dictionary from JSON url 
'''
def read_census_json(json_url): 
    
    # Make a GET request to the JSON url 
    response = requests.get(json_url)
    
    # Check if it was successful (status code == 200)
    if response.status_code == 200: 
        # This is the initial response, in dictionary form 
        json_dict = response.json()
        # There is only one key, so reassign the dictionary variable to that key 
        variables_dict = json_dict['variables']
    else: 
        print("Couldn't read JSON")
    
    print("JSON dictionary returned")
    
    return variables_dict

In [ ]:
'''
Rename columns
'''
def label_index_values(json_list, code_label_dictionary): 
    
    index_list = json_list[0][0]

    for i, index in enumerate(index_list): 

        try: 
            index_list[i] = code_label_dictionary[index]
        except: 
            pass 
    
    return index_list 

In [ ]:
def json_to_dictionary(json_list, index_list): 

    output_dictionary = {} 

    for element in json_list: 
        if element[1] == index_list: 
            for index in index_list: 
                output_dictionary[index] = [] 

        else: 
            for i, variable in enumerate(element[1]): 
                output_dictionary[index_list[i]].append(variable)

    return output_dictionary

In [ ]:
# PARAMETERS 
block_groups_fc_path = r"C:\DALE\Personal\Minnesota\MyProject\MyProject.gdb\Como_Park_Block_Groups"

In [ ]:
# Run functions 
api_key           = r"1749dc1d87964116107a80cd7d76fca300dda59f"
variable_string   = create_variable_string(variable_codes)
api_urls          = construct_api_urls(block_groups_fc_path, api_key, variable_string)
json_list         = call_census_api(api_urls)
index_list        = label_index_values(json_list, code_label_dictionary)

In [ ]:
census_dataframe  = json_list_to_dataframe(json_list, index_list)
census_dataframe  = change_columns_to_integers(census_dataframe)
census_dataframe  = calculate_percentage_columns(census_dataframe)

In [9]:
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

In [10]:
'''
Create variable string. 
'''

def create_variable_string(variable_codes): 
    
    # Initialize empty variable string 
    variable_string = r"NAME,B01001_001E"

    # Iterate through filtered dictionary 
    for code in variable_codes: 
        variable_string = variable_string + ',' + code
    
    return variable_string 

In [11]:
'''
Use the geographic field attributes of the feature classes to create a list of urls 
'''
def construct_api_urls(fc_path, api_key, variable_string): 
    
    api_urls = []
    
    with arcpy.da.SearchCursor(fc_path, ['STATE', 'COUNTY', 'TRACT', 'BLKGRP']) as cursor: 
        for row in cursor: 
            state       = row[0]
            county      = row[1] 
            tract       = row[2] 
            block_group = row[3] 

            var_clause    = fr"https://api.census.gov/data/2022/acs/acs5?get={variable_string}"
            geo_clause    = fr"&for=block%20group:{block_group}&in=state:{state}%20county:{county}%20tract:{tract}"
            key_clause    = fr"&key={api_key}"

            api_url       = var_clause + geo_clause + key_clause
            api_urls.append(api_url)
        
    return api_urls

In [12]:
'''
Iterate through API URLs and receive dictionary of responses 
'''

def call_census_api(api_urls):
    
    json_list = [] 
    url_count = 0
    
    # Initialize time 
    time_1  = time.time() 
    
    # Send API call 
    for url in api_urls: 
        url_count += 1 
        
        print(f'Call #{url_count} / {len(api_urls)}')
        response = requests.get(url)
        print(">>> Sent request")

        print("Checking status:")
        if response.status_code == 200: 
            json_list.append(response.json())
            print(">>> Good")

        else: 
            print('>>> Error: ', response.status_code)
            
        # Measure and print elapsed time 
        time_2  = time.time() 
        elapsed = round(time_2 - time_1, 2) 
        print(f'Elapsed time:')
        print(f">>> {elapsed}")
        print(" ")
    
    return json_list

In [13]:
def json_list_to_dataframe(json_list):

    output_dictionary = {} 
    
    index_list = json_list[0][0]
    
    for index in index_list: 
        output_dictionary[index] = []

    for json in json_list: 
        values = json[1]

        for i, value in enumerate(values): 
            output_dictionary[index_list[i]].append(value)

    df = pd.DataFrame.from_dict(output_dictionary)
    df = df.rename(columns = code_label_dictionary)
    
    df['GEOID'] = df['state'] + df['county'] + df['tract'] + df['block group']
    
    return df 

In [14]:
def change_columns_to_integers(dataframe): 
    
    for column in dataframe.columns: 

        try: 
            dataframe[column] = dataframe[column].astype(int)
        except: 
            pass
            
    return dataframe 

In [15]:
def calculate_percentage_columns(dataframe):

    for column in dataframe.columns: 
        if column in code_label_dictionary.values(): 
            column_position = dataframe.columns.get_loc(column) + 1
            percentage_formula = (dataframe[column]/dataframe['Total Population'])*100
            dataframe.insert(column_position, f'% {column}', round(percentage_formula, 2))
            
    return dataframe 